# Blackjack with AI player

In [264]:
import random

In [265]:
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, rank = None):
        self.rank = rank
        
        if not rank: # null card
            self.value = 0
            self.isAce = False
            return
        elif rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
            
    def __str__(self):
        return self.rank
    
    def __eq__(self, card):
        return self.value == card.value
    
    def __repr__(self):
        return self.rank


# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.cards = []
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        for i in range(nDecks):
            for j in range(4): # 4 of each card per deck
                for rank in RANKS:
                    self.cards.append(Card(rank))
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.bet = bet
        self.cards = []
        if card:
            self.cards.append(card)
        
    def addCard(self, card):
        self.cards.append(card)     
            
    def removeCard(self):
        return self.cards.pop()
    
    def isBust(self):
        if self.value > 21:
            return True
        return False
    
    def isBlackjack(self):
        if len(self.cards) == 2:
            if self.value == 21:
                return True
        return False
        
    def calculateValue(self, prop = 'value'):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        if (prop == 'aces'):
            return aces
        return value
        
    @property
    def value(self):
        return self.calculateValue()
    
    @property
    def soft(self):
        if self.calculateValue('aces'):
            return True
        return False
              
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
        
    def __repr__(self):
        return str(self.cards)

#### Testing Card, Deck, and Hand

In [266]:
ace = Card('A')
king = Card('K')
queen = Card('Q')
jack = Card('J')
ten = Card('10')
nine = Card('9')
eight = Card('8')
seven = Card('7')
six = Card('6')
five = Card('5')
four = Card('4')
three = Card('3')
two = Card('2')
noCard = Card()
passed = 0

# Test A,K,Q,9,4
if ace.value == 11:
    if ace.rank == 'A':
        passed += 1
if king.value == 10:
    if king.rank == 'K':
        passed += 1
if queen.value == 10:
    if queen.rank == 'Q':
        passed += 1
if four.value == 4:
    if four.rank == '4':
        passed += 1
if nine.value == 9:
    if nine.rank == '9':
        passed += 1
if noCard.value == 0:
        passed += 1

# Test Equality and Inequality
if king == queen:
    passed += 1 
if not king == nine:
    passed += 1
    
# All Tests
if passed == 8:
    print('All tests passed')
else:
    print(str(8-passed) + ' tests failed')

All tests passed


In [267]:
passed = 0
deck = Deck(1)

# Testing creating a deck and drawing cards from the deck
if len(deck) == 52:
    passed += 1
for i in range(5):
    deck.draw()
if len(deck) == 47:
    passed +=1
deck = Deck(6)
if len(deck) == 312:
    passed += 1
if passed == 3:
    print('All tests passed')
else:
    print(str(3-passed) + ' tests failed')

All tests passed


In [268]:
hand = Hand()
passed = 0

# Test empty hand
if len(hand) == 0:
    passed += 1
    
# Test hand.addCard()
hand.addCard(ace)
if len(hand) == 1:
    passed += 1
hand.addCard(nine)
if len(hand) == 2:
    passed += 1
    
# Test hand.isBust()
if not hand.isBust(): # no bust
    passed += 1
hand.addCard(king)
if not hand.isBust(): # no bust
    passed += 1
hand.addCard(ace)
hand.addCard(ace)
if hand.isBust():
    passed += 1

# Test hand.isBlackjack()
if not hand.isBlackjack(): # no Blackjack
    passed += 1
hand = Hand(ace)
hand.addCard(king)
if hand.isBlackjack(): # Blackjack
    passed += 1
hand = Hand(king)
hand.addCard(king)
if not hand.isBlackjack(): # no blackjack
    passed += 1
       
# Test hand.removeCard and hand.value
hand.removeCard()
hand.addCard(nine)
if hand.value == 19:
    passed += 1

if passed == 10:
    print('All tests passed')
else:
    print(str(10-passed) + ' tests failed')

All tests passed


## Player and Dealer

In [269]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000, bet = 50):
        self.money = money
        self.betAmount = bet
        self.bets = []
        self.hands = [Hand()]
        
    def makeBet(self, handNumber):
        if handNumber >= len(self.bets): # new hand gets a new bet
            self.bets.append(self.betAmount)
        else: # double the bet
            self.bets[handNumber] += self.betAmount
        # remove the bet amount from players money
        self.money -= self.betAmount
            
    def resetBets(self):
        self.bets = []
        
    # True if player has enough money to make a bet, False otherwise
    def hasMoney(self):
        return self.money > self.betAmount
        
    def hit(self, handNumber, card):
        self.hands[handNumber].addCard(card)

    def doubleDown(self, handNumber, card):
        self.hit(handNumber, card)
        self.makeBet(handNumber)

    def split(self, handNumber, cards):
        newHand = Hand()
        # remove one card from the hand being split and add to newHand
        newHand.addCard(self.hands[handNumber].removeCard())
        # add a second card to the old hand
        self.hands[handNumber].addCard(cards[0])  
        # add a second card to the new hand
        newHand.addCard(cards[1])
        # append newHand to players Hands and add bet for the hand
        self.hands.append(newHand)
        self.makeBet(len(self.hands)) # make a new bet for the new hand
        
        #print()
        #print('Post Split, Hand 1: ' + str(self.hands[handNumber]) + ' Hand 2: ' + str(newHand))
        #print('Bets: ' + str(self.bets))
        #print(self.hands)
        #print()
        
        
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print() 
        
    def __str__(self):
        return str(self.hands)

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            self.player.hands[0].addCard(self.deck.draw()) # Deal the player a card   
        self.player.makeBet(0) # player makes a bet on hand
       
    def play(self):
        while self.hand.value <= 17: # hits until they have 17
            # if soft 17, dealer must hit
            if self.hand.value == 17:
                if self.hand.soft:
                    self.hit()
                else:
                    break
            # less than 17, must hit
            else:
                self.hit()
                      
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def showHands(self):
        print('Dealers Hand:')
        print(self.hand)

#### Test Player and Dealer

In [270]:
passed = 0

# Test constructor
player = Player()
if len(player.hands[0]) == 0:
    passed += 1
    
# Test Player.makeBet()
player.makeBet(0)
if player.money == 4950:
    if player.bets[0] == 50:
        passed += 1
        
# Test Player.hasMoney()
if player.hasMoney():
    passed += 1
player.money = 0
if not player.hasMoney():
    passed += 1
player.money = 4950
    
# Tests passed
if passed == 4:
    print('All tests passed')
else:
    print(str(4-passed) + ' tests failed')

All tests passed


In [271]:
passed = 0
player = Player()
deck = Deck()
dealer = Dealer(deck, player)

# Test instantiation
if len(dealer.deck) == 52:
    passed += 1
if len(dealer.player.hands[0]) == 0:
    passed += 1
if len(dealer.hand) == 0:
    passed += 1
    
# Test dealer.deal()
dealer.deal()
if len(dealer.player.hands[0]) == 2:
    passed += 1
if len(dealer.hand) == 2:
    passed += 1
if len(dealer.deck) == 48:
    passed += 1
    
# Test dealer.hit()
dealer.hit()
if len(dealer.hand) == 3:
    passed += 1
if len(dealer.deck) == 47:
    passed += 1
    
# Test dealer.play()
deck = Deck()
player = Player()
dealer = Dealer(deck, player)
dealer.hand.addCard(ace)
dealer.hand.addCard(six)
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
dealer.play()
if len(dealer.hand) > 2:
    passed += 1
    
# Tests passed
if passed == 11:
    print('All tests passed')
else:
    print(str(11-passed) + ' tests failed')

All tests passed


## Valid Moves and Make

In [272]:
# returns a list of valid moves for a hand
def validMoves(player, handNumber):
    moves = ['hit','stand']
    # check that hand has 2 cards and the player has money to make a bet
    if len(player.hands[handNumber].cards) == 2:
        if player.hasMoney(): 
            moves.append('doubleDown')
            if player.hands[handNumber].cards[0].rank == player.hands[handNumber].cards[1].rank: # compare ranks same
                moves.append('split')
            
    return moves
    
def makeMove(game, handNumber, move):
    if move == 'hit':
        game.player.hit(handNumber, game.dealer.deck.draw())
        return False
    elif move == 'split':
        #print('Splitting: ' + str(game.player.hands[handNumber]))
        game.player.split(handNumber, [game.dealer.deck.draw(), game.dealer.deck.draw()])
        return False
    elif move == 'doubleDown':
        game.player.doubleDown(handNumber, game.dealer.deck.draw())
        return True
    else:
        return True



#### Testing ValidMoves and MakeMove

In [273]:
# Testing valid Moves
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(king)

# check all moves are avilable
moves = validMoves(player, 0)
if moves == ['hit','stand','doubleDown','split']:
    print('All moves valid: passed')
else:
    print('All moves valid: failed')
    print(moves)
    
# check no money moves
player = Player(0)
player.hands[0].addCard(king)
player.hands[0].addCard(king)
moves = validMoves(player, 0)
if moves == ['hit','stand']:
    print('No Money moves valid: passed')
else:
    print('No Money moves valid: failed')
    print(moves)
    
# check no split moves
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(five)
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown']:
    print('No split moves valid: passed')
else:
    print('No split moves valid: failed')
    print(moves)
    
# check basics - more than 2 cards
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(five)
player.hands[0].addCard(four)
moves = validMoves(player, 0)
if moves == ['hit','stand']:
    print('Basic moves 3 cards valid: passed')
else:
    print('Basic moves 3 valid: failed')
    print(moves)
    
# check split moves
player = Player()
player.hands[0].addCard(five)
player.hands[0].addCard(five)
moves = validMoves(player, 0)
if moves == ['hit','stand', 'doubleDown', 'split']:
    print('Split moves valid: passed')
else:
    print('Split moves valid: failed')
    print(moves)

All moves valid: passed
No Money moves valid: passed
No split moves valid: passed
Basic moves 3 cards valid: passed
Split moves valid: passed


In [274]:
player = Player()

# Test Split
player.hands[0].addCard(king)
player.hands[0].addCard(king)
player.bets.append(player.betAmount)
player.split(0, [five, two])
if player.hands[0].cards == [king, five]:   
    if player.hands[1].cards == [king, two]:
        if player.bets == [50,50]:
            print('split move: passed')
        else:
            print('split move: failed')
    else:
        print('split move: failed')
else:
    print('split move: failed')
    
# Test doubleDown
player = Player()
player.hands[0].addCard(king)
player.hands[0].addCard(two)
player.makeBet(0)
player.doubleDown(0, ten)
if player.hands[0].cards == [king, two, ten]:
    if player.bets == [100]:
        print('doubleDown move: passed')
    else:
        print('doubleDown move: failed')
else:
    print('doubleDown move: failed')
    
# Test hit
player.hit(0, ace)
if player.hands[0].cards == [king, two, ten, ace]:
    print('hit move: passed')
else:
    print('hit move: failed')    

split move: passed
doubleDown move: passed
hit move: passed


## Blackjack Game

In [275]:
class Blackjack:
    
    def __init__(self, player):
        self.player = player
        self.deck = Deck(6)
        self.dealer = Dealer(self.deck, player) # Remember, dealer holds the deck and deals, not the game
        self.dealer.deal()
        
    def newHand(self):
        self.player.hands = [Hand()]
        self.player.resetBets()
        self.dealer.hand = Hand()
        self.dealer.deal()
        
    def newDeck(self):
        self.dealer.deck = Deck(6)
       
    # hand is player's hand
    def gameStatus(self, hand):
        # player bust
        if hand.isBust():
            return -1
        # dealer bust
        if self.dealer.hand.isBust():
            return 1
        # draw
        if hand.value == self.dealer.hand.value:
            return 0
        # player win
        if hand.value > self.dealer.hand.value:
            return 1
        # dealer win
        if hand.value < self.dealer.hand.value:
            return -1
        
        

#### Test Blackjack

In [276]:
player = Player()
game = Blackjack(player)
passed = 0

def testBlackjack(game, decksize, money):
    passed = 0
    # Test player dealt correct number of cards, makes bet, and loses money coorectly
    if len(game.player.hands) == 1:
        if len(game.player.hands[0].cards) == 2:
            passed += 1
    if len(game.player.bets) == 1:
        if game.player.bets[0] == 50:
            passed += 1
    if game.player.money == money:
        passed += 1
    # Test dealer dealt correct number of cards
    if len(game.dealer.hand.cards) == 2:
        passed += 1
    # Test deck dealt correctly
    if len(game.dealer.deck) == decksize:
        passed += 1
    return passed

# Test initialization
passed += testBlackjack(game, 308, 4950)
# Test new hand
game.newHand()
passed += testBlackjack(game, 304, 4900)
# Test new deck
game.newDeck()
passed += testBlackjack(game, 312, 4900)

# Tests passed
if passed == 15:
    print('All tests passed')
else:
    print(str(15-passed) + ' tests failed')

All tests passed


In [277]:
player = Player()
game = Blackjack(player)
game.player.hands[0].removeCard()
game.player.hands[0].removeCard()
game.dealer.hand.removeCard()
game.dealer.hand.removeCard()
game.player.hands[0].addCard(ten)
game.player.hands[0].addCard(ten)
game.dealer.hand.addCard(ten)
game.dealer.hand.addCard(ten)
passed = 0

# Test player bust
game.player.hands[0].addCard(ten)
if game.gameStatus(game.player.hands[0]) == -1:
    passed += 1
game.player.hands[0].removeCard()

# Test dealer bust
game.dealer.hand.addCard(ten)
if game.gameStatus(game.player.hands[0]) == 1:
    passed += 1
game.dealer.hand.removeCard()

# Test push
if game.gameStatus(game.player.hands[0]) == 0:
    passed += 1

# Test player win
game.player.hands[0].addCard(ace)
if game.gameStatus(game.player.hands[0]) == 1:
    passed += 1
game.player.hands[0].removeCard()

# Test player loss
game.dealer.hand.addCard(ace)
if game.gameStatus(game.player.hands[0]) == -1:
    passed += 1
game.dealer.hand.removeCard()

# Tests passed
if passed == 5:
    print('All tests passed')
else:
    print(str(5-passed) + ' tests failed')


All tests passed


## TrainQ

In [278]:
import random
import numpy as np
import copy
import operator

# determines if a greedy move should be taken
def epsilonGreedy (epsilon, Q, player, handNumber, dealerCard, validMovesF):
    validMoves = validMovesF(player, handNumber)
    
    if np.random.uniform() < epsilon: # Random choice
        return random.choice(validMoves)
    else: # Greedy choice
        # Greedy Move
        Qs = np.array([Q.get(stateTuple(player.hands[handNumber], dealerCard, m), 0) for m in validMoves])
        return validMoves[ np.argmax(Qs) ]

def sortHand(hand):
    sortedHand = [card.rank for card in hand.cards]
    sortedHand.sort()
    return sortedHand
    
    
def stateTuple(hand, dealerCard, move):
    sortedHand = sortHand(hand)
    return (hand.value, str(dealerCard), move)

def updateQ(Q, movesMade, value, learningRate):
    for i, move in enumerate(movesMade):
        # If new move update Q table with 0
        if move not in Q:
            Q[move] = 0
            
        if i == 0:
            Q[move] += learningRate*(value + Q[move])
        elif i < len(movesMade)-1:
            # Update the move with learning rate
            Q[move] += learningRate*(value + Q[move] - Q[movesMade[i-1]])
        else:
            Q[move] += value
    return Q

In [279]:
hand = Hand(ace)
hand.addCard(king)
hand.addCard(ten)
hand.addCard(nine)
hand.addCard(seven)
hand.addCard(two)
hand.addCard(three)
hand.addCard(queen)
hand.addCard(four)
hand.addCard(six)
hand.addCard(five)
hand.addCard(jack)
hand.addCard(eight)
print(hand)
print(stateTuple(hand, queen, 'stand'))

['A', 'K', '10', '9', '7', '2', '3', 'Q', '4', '6', '5', 'J', '8']
(85, 'Q', 'stand')


In [280]:

def trainQ(nRepetitions, learningRate, epsilonDecayRate, validMovesF, makeMoveF):
    epsilon = 1.0
    
    Q = {}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    
    for gameNum in range(nRepetitions):
        if gameNum%100 == 0:
            print(gameNum)
        epsilon *= epsilonDecayRate  # decay epsilon to move away from random choices
        epsilons[gameNum] = epsilon
        
        # create a game
        player = Player()
        blackjack = Blackjack(player)
        
        done = False
        
        # play some blackjack
        while len(blackjack.deck) >  52: # deck is 6 decks, last deck is a buffer deck
            if blackjack.player.hasMoney(): # make sure player has the funds to play
                blackjack.newHand() # deal a hand
                handNumber = 0                
            
                # For each players hand
                while handNumber < len(blackjack.player.hands):
                    done = False
                    
                    #print(blackjack.player.hands[handNumber])
                    # Check for a blackjack
                    if blackjack.player.hands[handNumber].isBlackjack():
                        #print()
                        #print('Blackjack')
                        #print(blackjack.player.hands)
                        #print('hand number: ' + str(handNumber))
                        #print('bets: ' + str(player.bets))
                        #print()
                        blackjack.player.money += 2*blackjack.player.bets[handNumber] + .5*blackjack.player.bets[handNumber]
                        done = True
                    
                    movesMade = [] # holds the moves made on this hand
                    step = 0
                    
                    # Player plays each hand until they stand or bust
                    while not done:
                        step += 1
                        
                        # Determine a move for the players hand
                        move = epsilonGreedy(epsilon, Q, blackjack.player, handNumber, game.dealer.hand.cards[1], validMovesF)
                        
                        newGame = copy.deepcopy(blackjack)
                        done = makeMoveF(newGame, handNumber, move)
                        
                        # if new move, add to Q
                        if stateTuple(blackjack.player.hands[handNumber], blackjack.dealer.hand.cards[1], move) not in Q:
                            Q[stateTuple(blackjack.player.hands[handNumber], blackjack.dealer.hand.cards[1], move)] = 0                        
                        
                        # if move results in 21, set Q value = to 0
                        if newGame.player.hands[handNumber].value == 21:
                            done = True
                        # if move results in bust, update Q with reinforcement
                        elif newGame.player.hands[handNumber].isBust():
                            done = True
        
                        # store a tuple (hand, dealercard, move) for later, player can only 'see' one of dealers cards
                        movesMade.append(stateTuple(blackjack.player.hands[handNumber], blackjack.dealer.hand.cards[1], move))
                        
                        blackjack = copy.deepcopy(newGame)
                            
                    handNumber += 1 # next hand
                    
                # After Player plays all his hands dealer plays
                blackjack.dealer.play()
                #print('moves: ' + str(movesMade))
                
                # After Dealer plays, check all players hands and payout on wins
                for hand,nothing in enumerate (blackjack.player.hands):
                    handStatus = blackjack.gameStatus(blackjack.player.hands[hand])
                    # update Q table
                    Q = updateQ(Q, movesMade, handStatus, learningRate)
                    if handStatus == 0: # tie (push)
                        blackjack.player.money += blackjack.player.bets[hand] # player gets their money back
                    if handStatus == 1: # win
                        blackjack.player.money += 2*blackjack.player.bets[hand] # player gets their money plus winnings
                        
            else:
                break # player doesn't have the funds, game over 
        # update outcome
        outcomes[gameNum] = blackjack.player.money
    return Q, outcomes, epsilons
        
    
    # test trainQ
    # sort hands before tuple
    # update Q - only last one
 
    
def testQ(Q, maxSteps , validMovesF, makeMoveF):
    
    player = Player(maxSteps*55)
    blackjack = Blackjack(player)
    
    playBlackjack(Q, blackjack, maxSteps, validMovesF, makeMoveF)
        
    return (maxSteps*55, blackjack.player.money)
        
def playBlackjack(Q, blackjack, numberHands, validMovesF, makeMoveF): 
    for i in range(numberHands):
        if i%100 == 0:
            print(i)
        if len(blackjack.dealer.deck) < 52:
            blackjack.newDeck()
            
        blackjack.newHand() # deal a hand
        handNumber = 0     
        while handNumber < len(blackjack.player.hands):
            done = False
            
            if blackjack.player.hands[handNumber].isBlackjack():
                blackjack.player.money += 2*blackjack.player.bets[handNumber] + .5*blackjack.player.bets[handNumber]
                done = True
            
            while not done: 
                validMoves = validMovesF(blackjack.player, handNumber)
                Qs = np.array([Q.get(stateTuple(blackjack.player.hands[handNumber], dealer.hand.cards[1], m), 0) for m in validMoves])
                move = validMoves[ np.argmax(Qs) ] # choose move
                done = makeMoveF(blackjack, handNumber, move)
                if blackjack.player.hands[handNumber].value == 21:
                    done = True
                if blackjack.player.hands[handNumber].isBust():
                    done = True
            handNumber += 1
                
                
        # total hands
        for hand,nothing in enumerate (blackjack.player.hands):
            handStatus = blackjack.gameStatus(blackjack.player.hands[hand])
            if handStatus == 0: # tie (push)
                blackjack.player.money += blackjack.player.bets[hand] # player gets their money back
            if handStatus == 1: # win
                blackjack.player.money += 2*blackjack.player.bets[hand] # player gets their money plus winnings
    

## Testing

In [281]:

Q,outcomes,_ = trainQ(2500, 0.1, 0.4, validMoves, makeMove)
    
print(outcomes)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
[ 4500.  3825.  4450. ...,  4500.  4250.  4450.]


In [282]:
print(Q)

{(12, 'J', 'hit'): 2.4547522904925292e+39, (11, '7', 'stand'): 0.669, (15, 'J', 'hit'): -38458265.59287524, (20, '9', 'hit'): -0.1, (20, '6', 'hit'): -0.21000000000000002, (15, 'A', 'hit'): 3.0918906716850026e+21, (16, '10', 'hit'): -1.5408383166694726e+35, (7, '9', 'hit'): -40.98718796587971, (12, '9', 'hit'): -2.2031555671251323e+38, (17, '9', 'hit'): -44190191.799459234, (17, '10', 'hit'): -5.2720956492802463e+23, (14, '10', 'hit'): -8.363174344030614e+24, (11, '4', 'stand'): 0.063361, (20, '3', 'doubleDown'): 0.1, (6, '5', 'hit'): 124.38311206951784, (17, '5', 'stand'): 10375110763700.928, (11, 'Q', 'hit'): 10772.48772957405, (15, 'Q', 'stand'): -4392227051378963.5, (17, 'J', 'stand'): -1528751609515.8855, (13, '2', 'stand'): -1.400723751080845e+17, (20, 'K', 'hit'): -0.1, (12, '10', 'stand'): -0.4641, (17, 'Q', 'stand'): -8022087521791.095, (13, 'Q', 'hit'): 821328798.4434245, (16, 'Q', 'stand'): -7681763566123.111, (19, 'K', 'hit'): -3.9, (19, '4', 'hit'): -0.1, (17, '4', 'stand'

In [283]:
print(testQ(Q, 1000 , validMoves, makeMove))

0
100
200
300
400
500
600
700
800
900
(55000, 64800.0)


In [284]:
hand = Hand(eight)
hand.addCard(two)
dealerCard = '7'
Qs = np.array([Q.get(stateTuple(hand, dealerCard, m), 0) for m in ['hit', 'stand', 'split', 'doubleDown']])
print(Qs)

# change dealer card to value
# change player to value (it will work but how can we tell what was in the hand)

[ -3.67332044e+15   0.00000000e+00   0.00000000e+00  -1.00000000e-01]


In [285]:
deck = Deck(1)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

52
['2', '9', '6', '6', '7', '4', '8', 'A', 'J', '3', '3', 'Q', '9', 'Q', '7', 'K', 'K', '10', '10', 'A', 'Q', '8', '3', '10', 'J', '10', '2', '3', '6', '9', '2', 'K', '7', 'A', '5', '4', 'Q', 'J', '5', '7', '4', 'A', '8', '9', '2', '5', 'K', '4', '8', 'J', '5', '6']
2
9
50


In [286]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.value))

156
['J']
['J', 'Q']
['J', 'Q', '10']
30


In [287]:
player = Player()
game = Blackjack(player)
print(len(game.Deck))
game.Player.showHands()
game.Dealer.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.showHands()
game.newHand()
print(len(game.Deck))
game.Player.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.hit()
game.Dealer.showHands()
game.newDeck()
print(len(game.Deck))

AttributeError: 'Blackjack' object has no attribute 'Deck'

In [ ]:
game.gameStatus(game.Player.hands[0])

#### Testing validMoves and makeMove